In [27]:
import os
import sys
from pathlib import Path

import pandas as pd
import numpy as np
import plotly.express as px

In [28]:
df_rs1 = pd.read_feather('../datasets/rings_server1.feather')
df_rt2 = pd.read_feather('../datasets/rings_tools2.feather')

In [29]:
df_rs1.head()

,ip,identd,userid,datetime,request,status,size,referer,user_agent,date,month_year,DOW,hour
0,66.249.79.92,None,None,2021-01-01 08:00:12+00:00,GET /pds4/bundles/cassini_vims_saturn/data_raw...,200,88064.0,None,Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Bu...,2021-01-01,01-2021,Friday,8
1,66.249.79.92,None,None,2021-01-01 08:00:16+00:00,GET /pds4/bundles/cassini_vims_saturn/data_raw...,200,599040.0,None,Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Bu...,2021-01-01,01-2021,Friday,8
2,114.119.138.239,None,None,2021-01-01 08:00:16+00:00,GET /press_releases/pages/PIA08xxx/PIA08204.ht...,200,18417.0,None,Mozilla/5.0 (Linux; Android 7.0;) AppleWebKit/...,2021-01-01,01-2021,Friday,8
3,5.255.253.135,None,None,2021-01-01 08:00:19+00:00,GET /press_releases/pages/PIA18xxx/PIA18360.ht...,200,18711.0,None,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,2021-01-01,01-2021,Friday,8
4,13.66.139.95,None,None,2021-01-01 08:00:52+00:00,GET /saturn/cassini/jpegMod/PIA18271_modest.jp...,200,97957.0,None,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,2021-01-01,01-2021,Friday,8


In [30]:
df_rs1.describe()

,status,size,hour
count,3.465553e+06,3.247864e+06,3.465553e+06
mean,2.257778e+02,4.958414e+06,1.250906e+01
std,6.331875e+01,1.717696e+08,7.041412e+00
min,2.000000e+02,1.000000e+00,0.000000e+00
25%,2.000000e+02,2.503000e+03,6.000000e+00
50%,2.000000e+02,1.737200e+04,1.300000e+01
75%,2.000000e+02,1.914100e+04,1.900000e+01
max,5.040000e+02,5.029008e+10,2.300000e+01


In [ ]:
fig = px.line(df_rs1, x='datetime', y='size')

In [17]:
df_rs1['datetime'].dt.to_period('M')

/Users/kaipak/miniconda3/envs/dask/lib/python3.10/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



0          2021-01
1          2021-01
2          2021-01
3          2021-01
4          2021-01
            ...   
3465548    2021-07
3465549    2021-07
3465550    2021-07
3465551    2021-07
3465552    2021-07
Name: datetime, Length: 3465553, dtype: period[M]

In [19]:
df_rs1['datetime'][:10].dt.strftime('%m-%Y')

0    01-2021
1    01-2021
2    01-2021
3    01-2021
4    01-2021
5    01-2021
6    01-2021
7    01-2021
8    01-2021
9    01-2021
Name: datetime, dtype: object

In [21]:

df_rs1['datetime'][:-10].dt.strftime('%w-%Y')

0          5-2021
1          5-2021
2          5-2021
3          5-2021
4          5-2021
            ...  
3465538    5-2021
3465539    5-2021
3465540    5-2021
3465541    5-2021
3465542    5-2021
Name: datetime, Length: 3465543, dtype: object

In [26]:
df_rs1['datetime'].dt.date

0          2021-01-01
1          2021-01-01
2          2021-01-01
3          2021-01-01
4          2021-01-01
              ...    
3465548    2021-07-16
3465549    2021-07-16
3465550    2021-07-16
3465551    2021-07-16
3465552    2021-07-16
Name: datetime, Length: 3465553, dtype: object